In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e9\train.csv")
test = pd.read_csv(r"C:\Users\Shraddha\Downloads\playground-series-s5e9\test.csv")

In [3]:
train.head()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989
3,3,0.734463,-1.357000,0.052965,0.001651,0.159717,0.086366,0.278745,225567.4651,0.487467,147.91212
4,4,0.532968,-13.056437,0.023500,0.068687,0.000001,0.331345,0.477769,213960.6789,0.947333,89.58511


In [4]:
train = train.drop(columns=['id'])

In [5]:
train.head()

,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989
3,0.734463,-1.357000,0.052965,0.001651,0.159717,0.086366,0.278745,225567.4651,0.487467,147.91212
4,0.532968,-13.056437,0.023500,0.068687,0.000001,0.331345,0.477769,213960.6789,0.947333,89.58511


In [6]:
X = train.drop(columns=['BeatsPerMinute'])
y = train["BeatsPerMinute"]

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
test_ids = test['id']
test_features = test.drop(columns=['id'])

In [9]:
X = scaler.fit_transform(X)
test_features = scaler.transform(test_features)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=20,
    n_jobs=-1,
    random_state=42
)

In [16]:
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [17]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print(rmse)

26.462663153634686


In [22]:
test_preds = rf.predict(test_features)
PredictingBeats = pd.DataFrame({"id": test_ids, "BeatsPerMinute": test_preds})
PredictingBeats.to_csv("PredictingBeats.csv", index=False)

In [12]:
import lightgbm as lgb

In [55]:
model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    max_depth=10,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.2,
    n_jobs=-1,
    random_state=42
)

In [56]:
model.fit(X_train,y_train,eval_set=[(X_test,y_test)],eval_metric='rmse',callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 419331, number of used features: 9
[LightGBM] [Info] Start training from score 119.056554
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 26.4406	valid_0's l2: 699.104
Early stopping, best iteration is:
[47]	valid_0's rmse: 26.4394	valid_0's l2: 699.044


,boosting_type,'gbdt'
,num_leaves,63
,max_depth,10
,learning_rate,0.03
,n_estimators,2000
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [57]:
y_pred = model.predict(X_test)

c:\Users\Shraddha\OneDrive\Desktop\forecast_alerting\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [58]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print(rmse)

26.43943640223121


In [24]:
test_preds = model.predict(test_features)
submission = pd.DataFrame({"id": test_ids, "BeatsPerMinute": test_preds})
submission.to_csv("PredictingBeats_LightGBM.csv", index=False)

c:\Users\Shraddha\OneDrive\Desktop\forecast_alerting\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
